# Deploy the trained Model and invoke it using Exasol

In this part of the tutorial we will show how to deploy and invoke the trained model using Exasol. For this we will show two versions.
You can either:


   * [Deploy the model using an AzureML online Endpoint and then invoke it via an Exasol UDF](#deploy-the-model-in-an-azureml-endpoint)

Or:
   * [Load the model into Exasols Filesystem (BucketFS), and then deploy and invoke it via an Exasol UDF](#deplay-and-invoke-the-model-in-exasol)

Which version you choose is up to you.


## Deploy the model in an AzureML endpoint

In this Section we will explain how to Deploy the model in an AzureML endpoint, and the invoke it via an Exasol UDF

### Prerequisites
 ## TODO
 * AzureML
 * Compute
 * trained and registered model
 * exasol with dataset

## Create online endpoint in azureml

First we need to set up an online endpoint with our trained and registered model in AzureML, so we can use it for real time inferencing. We will do this using Python in an AzureML Notebook. You can find the AzureML tutorial for this [here](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=python)


Load this notebook into your AzureML Notebooks and start your Compute. Then we can install some dependencies if not already installed from previous steps.

In [ ]:
!pip install azure-identity
!pip install azure-ai-ml==1.3.0

Import the required libraries

In [319]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

ModuleNotFoundError: No module named 'azure'

Now we can enter our Credentials to access our workspace.

In [ ]:
credential = DefaultAzureCredential()
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<your subscription id>",               # change
    resource_group_name="<your resource group name>",       # change
    workspace_name="<your workspace name>",                 # change
)

Now we are set up to start to set up our online endpoint we then want to deploy our model in. We set it up with key authentication, but you could use token authentication instead.

In [ ]:
# Define an endpoint name
endpoint_name = "<your-endpoint-name>"                      # change

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name,
    description="<some description>",                       # change
    auth_mode="key"
)

Create the Endpoint.

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint)

### Make a Deployment for the model

We now need to Deploy our model to the Endpoint via an AzureML Deployment. For this we need an Environment definition which can run our model. This need to include the "azureml-inference-server-http" package. There are also ready made images available from Microsoft, but here we make our own.
We take the conda file that is saved in our registered MLflow model and edit it to include the "azureml-inference-server-http" package. Then write it to a file.
#### (TODO explain how find?)

In [ ]:
%%writefile ./conda.yml
channels:
- conda-forge
dependencies:
- python=3.8.16
- pip<=23.1.2
- pip:
  - azureml-inference-server-http
  - mlflow==1.26.1
  - cloudpickle==2.2.1
  - scikit-learn==1.0.2
name: endpoint-env

Now we get a handle to our registered model and create the Environment for our Deployment.

In [ ]:
from azure.ai.ml.constants import AssetTypes

model = ml_client.models.get(name="<your registered model name>", version="<version of your model you want to use>")  # change

env = Environment(
    name="<name the Environment>",                                      # change
    description="Custom environment for azureML tutorial endpoint",
    conda_file="./conda.yml",                                           # change if necessary, path to your conda.yaml fil we created earlier
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest", # base image from microsoft we use
)

With this we now can create our Deployment.
The Deployment uses a [scoring script](score.py). This script has an "init()" function which loads the model, and a "run" function which takes the input data and feeds it to the model. This function returns the classification results. Make sure you have the scoring script in your AzureML files.

In [320]:

cc = CodeConfiguration(code=".", scoring_script="score.py")     # change if necessary to point to your scoring script in AzureML
model_deployment = ManagedOnlineDeployment(
    name="<name your deployment>",          # change
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=cc,
    instance_type="Standard_DS1_v2",        # Type of Azure Instance. You can change this if you need more computing power
    instance_count=1,
)

NameError: name 'CodeConfiguration' is not defined

Now we can create the Deployment on our endpoint.

In [ ]:
ml_client.online_deployments.begin_create_or_update(model_deployment)

You can check the status of your Deployment by geting the logs with this comand. Alternativly you can also Navigate to your Deployment in AzureML in your Browser and chack status and logs there.
### todo screenshots?

In [ ]:
ml_client.online_deployments.get_logs(
    name="<your deployment name>", endpoint_name=endpoint_name, lines=50
)

In order to access this Endpoint we will need a key (or a token if you choose to go with token authentication above). Get the key like this:

In [ ]:
endpoint_key = ml_client.online_endpoints.get_keys(name=endpoint_name)
print(endpoint_key.primary_key)

### Invoke the Endpoint from Exasol

In order to invoke our deployed Endpoint, we first need access to our Exasol Saas Database. We will use pyExasol for this.
### todo link

In [7]:
!pip install pyexasol
import pyexasol
EXASOL_HOST = "<your>.clusters.exasol.com"      # change
EXASOL_PORT = "8563"                            # change if needed
EXASOL_USER = "<your-exasol-user>"              # change
EXASOL_PASSWORD = "exa_pat_<your_password>"     # change
EXASOL_SCHEMA = "IDA"                           # change if needed

EXASOL_CONNECTION = f"{EXASOL_HOST}:{EXASOL_PORT}"

You should consider upgrading via the '/home/marlene/PycharmProjects/data-science-examples/venv/bin/python -m pip install --upgrade pip' command.


In [252]:
exasol = pyexasol.connect(dsn=EXASOL_CONNECTION, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)

We will now use a Python UDF(link?) to call the AzureML Online Endpoint we just created and use it to classify out test data we loaded into Exasol Saas in the first part of this tutorial(link).
Take care to only use the supported(link) packages, or build your own language container(link). The udf takes the data in the provided table and sends is as a REST request to the Endpoint. The UDF will then output the returned result.

### TODO links
explain where find deployment url

Change the endpoint key (will be different for each time you create the endpoint), the scoring url and the deployment name to access your endpoint.

In [194]:
sql_open_schema = """OPEN SCHEMA "IDA";"""

sql_create_invoke_endpoint ="""
CREATE OR REPLACE PYTHON3 SET SCRIPT IDA.invoke_endpoint(...)
EMITS ("ID" DECIMAL(20,0), "result" VARCHAR(200)) AS

import requests
import ujson
import pandas as pd
import numpy as np
import time
import re


def run(ctx):
    # set up needed info to send the request
    endpoint_key = "<your endpoint key>"                                # change
    deployment_name = "<your deployment name>"                          # change
    headers = {'Content-Type':'application/json',
                'Authorization': f'Bearer {endpoint_key}',
               'azureml-model-deployment': f'{deployment_name}'}
    scoring_url = "<scoring url>"                                       # change


    while True:
        # get the data
        try:
            df = ctx.get_dataframe(500)
        except:
            return 0
        if df is None:
            break

        # remove the label from the test data
        id_column = df["0"]
        df = df.drop("0", 1)

        # cast data to floats and switch "nan" to "null" so it will encode and decode as a valid json
        df = df.apply(pd.to_numeric, downcast='float', errors='ignore')
        df_str = str((df.values).tolist())
        df_str = df_str.replace("nan", 'null')

        PARAMS = '{"data": ' + df_str + '}'

        # send the request
        try:
            result = requests.post(url=scoring_url, data=PARAMS, headers=headers)
        except:
            return 0

        str_l = re.findall('[0,1]', result.text)
        df_res = pd.DataFrame(str_l)
        ctx.emit(pd.concat([id_column, df_res], axis=1))
/
"""

exasol.execute(sql_open_schema)
exasol.execute(sql_create_invoke_endpoint)


<ExaStatement session_id=1778269931084447746 stmt_idx=2>

Now we need to select the columns we want to send to the Endpoint. These should be the same we used to train the endpoint in the part of the tutorial(link).
### TODO link

In [195]:
column_names = ['AA_000', 'AG_005', 'AH_000', 'AL_000', 'AM_0', 'AN_000', 'AO_000', 'AP_000', 'AQ_000',
                    'AZ_004', 'BA_002', 'BB_000', 'BC_000', 'BD_000', 'BE_000',
                    'BF_000', 'BG_000', 'BH_000', 'BI_000', 'BJ_000', 'BS_000', 'BT_000', 'BU_000', 'BV_000',
                    'BX_000', 'BY_000', 'BZ_000', 'CA_000', 'CB_000', 'CC_000', 'CI_000', 'CN_004', 'CQ_000',
                    'CS_001', 'DD_000', 'DE_000', 'DN_000', 'DS_000', 'DU_000', 'DV_000', 'EB_000', 'EE_005']

That is all we need to call our UDF which will call the Endpoint from within Exasol and the return te classification results to us. The call also adds a ROWID to be able to sort the result to the right input, so we can then check how many where correctly classified by our endpoint.


In [196]:
res = exasol.export_to_pandas("""SELECT "CLASS", "result" FROM  (
                           SELECT IDA.invoke_endpoint(ROWID, {columns!q}) FROM IDA.TEST t) r
                           JOIN IDA.TEST o ON r.ID = o.ROWID""", {"columns": column_names})

And now we can create a confusion matrix from our results.

In [197]:
import pandas as pd
pd.crosstab(index=res['CLASS'], columns=res["result"], rownames=['actuals'], colnames=['predictions'])

predictions,0,1
actuals,,
neg,14841,784
pos,13,362


Don’t forget to Delete the endpoint once you don’t need it anymore, and also close your pyexasol connection.

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)

In [ ]:
exasol.close()

## Deploy and invoke the model in Exasol

Let's look into how to deploy and invoke the model directly on the Exasol Saas Cluster.

### Prerequisites

* trained model
* Exasol Saas with data
* PyExasol(link)


First we need to download the model from AzureML and upload it into an Exasol Bucket in the BucketFS (Exasol's built-in Filesystem).
## todo explain how download
Then we can upload it to the BucketFS using the online Saas interface.
# pics and explanation

### todo Check if uploaded, get path from Saas, pics

Then we will use PyExasol to connect to our Exasol Saas Cluster.

In [ ]:
!pip install pyexasol
import pyexasol
import pandas as pd

EXASOL_HOST = "<your>.clusters.exasol.com"      # change
EXASOL_PORT = "8563"                            # change if needed
EXASOL_USER = "<your-exasol-user>"              # change
EXASOL_PASSWORD = "exa_pat_<your_password>"     # change
EXASOL_SCHEMA = "IDA"                           # change if needed

# get the connection
EXASOL_CONNECTION = f"{EXASOL_HOST}:{EXASOL_PORT}"

make sure your ip is whitelisted in exasol saas

In [193]:
exasol = pyexasol.connect(dsn=EXASOL_CONNECTION, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)

Then, move model from AzureML to Exasol BucketFS.
open saas, start db, clck manage udf files, click new de, upload file (only works in enterprise edition)
copy path of file

## call and invoke using udf

In [2]:
sql_open_schema = """OPEN SCHEMA "IDA";"""
sql_ls = """ --/
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT "IDA.LS" ("my_path" VARCHAR(100)) EMITS ("FILES" VARCHAR(100)) AS
import os
def run(ctx):
    for line in os.listdir(ctx.my_path):
        ctx.emit(line)
/
"""

sgl_run_ls = """SELECT "IDA.LS"('/buckets/uploads/default/testfolder');"""

exasol.execute(sql_open_schema)

<ExaStatement session_id=1774377255978401792 stmt_idx=1>

In [15]:
exasol.execute(sql_ls)

<ExaStatement session_id=1774293578104963073 stmt_idx=16>

In [3]:
exasol.export_to_pandas(sgl_run_ls)

,FILES
0,model.pkl


We will now use a Python UDF(link?) to load the model and use it to classify out test data we loaded into Exasol Saas in the first part of this tutorial(link). Take care to only use the supported(link) packages, or build your own language container(link). The udf takes the data in the provided table and uses the model to classify it. The UDF will then output the result.

## todo chang to set script?

In [38]:
sql_create_inference_udf = """
--/
CREATE OR REPLACE PYTHON3 SET SCRIPT IDA.use_model_for_inference(...)
EMITS ("ID" DECIMAL(20,0), "prediction" DOUBLE) AS

import urllib.request
import lxml.etree as etree
import sklearn
import numpy
import pickle
import pandas as pd

def load_model():
    model_path = "/buckets/uploads/default/testfolder/model.pkl"  # change to your model file path
    # deserialize the model file back into a sklearn model
    model = pickle.load(open(model_path, 'rb'))
    return model

def infer(data, model):

    data = numpy.array(data)
    result = model.predict(data)
    return result


def run(ctx):
    model = load_model()
    while True:
        df = ctx.get_dataframe(num_rows=1000)
        if df is None:
            break
        id_column = df["0"]
        df = df.drop("0", 1)
        response = infer(df, model)
        result = pd.DataFrame(response)
        ctx.emit(pd.concat([id_column,result],axis=1))

/
"""
exasol.execute(sql_create_inference_udf)

<ExaStatement session_id=1778190569628434435 stmt_idx=33>

Now we need to select the columns we want to send to the Endpoint. These should be the same we used to train the endpoint in the part of the tutorial(link).
### TODO link

In [9]:
column_names = ['AA_000', 'AG_005', 'AH_000', 'AL_000', 'AM_0', 'AN_000', 'AO_000', 'AP_000', 'AQ_000',
                    'AZ_004', 'BA_002', 'BB_000', 'BC_000', 'BD_000', 'BE_000',
                    'BF_000', 'BG_000', 'BH_000', 'BI_000', 'BJ_000', 'BS_000', 'BT_000', 'BU_000', 'BV_000',
                    'BX_000', 'BY_000', 'BZ_000', 'CA_000', 'CB_000', 'CC_000', 'CI_000', 'CN_004', 'CQ_000',
                    'CS_001', 'DD_000', 'DE_000', 'DN_000', 'DS_000', 'DU_000', 'DV_000', 'EB_000', 'EE_005']

We can now call the UDF to predict on our test data using the model we trained in Exasol. The call also adds a ROWID to be able to sort the result to the right input, so we can then check how many where correctly classified by our endpoint.

In [39]:
res = exasol.export_to_pandas("""SELECT "CLASS", "prediction" FROM  (
                           SELECT IDA.use_model_for_inference(ROWID, {columns_without_class!q}) FROM IDA.TEST t) r
                           JOIN IDA.TEST o ON r.ID = o.ROWID""", {"columns_without_class": column_names})

And now we can create a confusion matrix from our results.

In [25]:
import pandas as pd

pd.crosstab(index=res['CLASS'], columns=res["prediction"], rownames=['actuals'], colnames=['predictions'])

predictions,0,1
actuals,,
neg,14841,784
pos,13,362


Don't forget to close your connection.

In [ ]:
exasol.close()